# 8.1 维度的诅咒

* 高维数据集很大可能是非常稀疏的：大多数训练实例可能彼此之间相距很远
* 训练集的维度越高，过拟合的风险就越大

# 8.2 降维的主要方法

## 8.2.1 投影

所有训练实例都位于(或接近于)高维空间的低维子空间内

投影并不只是降低尺寸的最佳方法。在许多情况下，子空间可能会发生扭曲和转动。

## 8.2.2 流形学习

通过对训练实例所在的流形进行建模  
依赖于流形假设：大多数现实世界的高维数据集都接近于低维流形  
在训练模型之前降低训练集的维度肯定可以加快训练速度，但这并不总是会导致更好或更简单的解决方案

# 8.3 PCA

最流行的降维算法  
首先识别最靠近数据的超平面，然后将数据投影到其上

## 8.3.1 保留差异性

将训练集投影到低维超平面之前需要选择正确的超平面  
选择保留最大差异性的轴看起来比较合理  
比较原始数据集与其轴上的投影之间的均方距离，使这个最小的轴是最合理的选择

## 8.3.2 主要成分

主成分分析可以在训练集中识别出哪条轴对差异性的贡献度最高   
第i个轴称为数据的第i个主要成分(PC)  
对于每个主要成分，PCA都找到一个指向PC方向的零中心单位向量

奇异值分解(SVD): 
将训练集矩形$X$分解为三个矩阵${U∑V}^T$的矩阵乘法，其中$V$包含定义所有主要成分的单位向量

In [1]:
# 使用NumPy的svd()函数来获取训练集的所有主要成分，然后提取定义前两个PC的两个单位向量

import numpy as np
from sklearn.datasets import make_swiss_roll

X, t = make_swiss_roll(n_samples=2000, noise=0.1)

X_centered = X - X.mean(axis=0)     # 居中数据
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]


## 8.3.3 向下投影到d维度

确定所有主要成分后，将数据集投影到前d个主要成分定义的超平面上，从而将数据集的维度降低到d维

将训练集投影到d维度
$$X{_{d-proj}}=XW{_d}$$  
矩阵$W_d$定义为包含$V$的前$d$列的矩阵

In [2]:
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

## 8.3.4 使用Scikit-Learn

In [3]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

将PCA转换器拟合到数据集后，其components_属性是$W_d$的转置  
定义第一个主成分的单位向量等于pca.components_.T[:, 0]

## 8.3.5 可解释方差比

In [4]:
# 通过explained_variance_ratio_变量来获得
# 该比率表示每个成分的数据集方差的比率

pca.explained_variance_ratio_

array([0.40036305, 0.31328611])

数据集方差的40%位于第一个PC上，而31%位于第二个PC上  
对于第三个PC，不到29%，因此第三个PC携带的信息很少

## 8.3.6 选择合适的维度

In [5]:
# 在不降低维度的情况下执行PCA, 然后计算保留95%训练集方差所需的最小维度：
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

In [6]:
# 可以设置n_components=d并再次运行PCA
# 或将n_components设置为0.0到1.0之间的浮点数来表示要保留的方差率，而不是指定要保留的主成分数：

pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

## 8.3.7 PCA压缩

原始数据与重构数据(压缩后再解压缩)之间的均方距离称为重构误差

In [7]:
# 将MNIST数据集压缩为154个维度，然后使用inverse_transform()方法将其解压缩回784个维度

from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
X_train = mnist["data"]

pca = PCA(n_components=154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)


## 8.3.8 随机PCA

快速找到前d个主成分的近似值

In [8]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_train)

# 默认设置为"auto", 如果m或n大于500并且d小于m或n的80%, 则Scikit-Learn自动使用随机PCA算法

## 8.3.9 增量PCA

把训练集划分为多个小批量，并一次将一个小批量送入IPCA算法

In [9]:
# 将MNIST数据集拆分为100个小批量
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

# 8.4 内核PCA

内核:可以将实例隐式映射到一个高维空间(称为特征空间)，从而可以使用支持向量机来进行非线性分类和回归。

应用于PCA，从而可以执行复杂的非线性投影来降低维度

In [10]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

#### 选择内核并调整超参数

In [12]:
# 创建一个两部流水线，首先使用kPCA将维度减少到二维，然后使用逻辑回归来分类
# 使用GridSearchCV来查找kPCA的最佳内核和gamma值

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])

param_grid = [{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf", "sigmoid"]
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, t.astype('int'))

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kpca', KernelPCA(n_components=2)),
                                       ('log_reg', LogisticRegression())]),
             param_grid=[{'kpca__gamma': array([0.03      , 0.03222222, 0.03444444, 0.03666667, 0.03888889,
       0.04111111, 0.04333333, 0.04555556, 0.04777778, 0.05      ]),
                          'kpca__kernel': ['rbf', 'sigmoid']}])

In [13]:
print(grid_search.best_params_)

{'kpca__gamma': 0.03, 'kpca__kernel': 'sigmoid'}


# 8.5 LLE

非线性降维技术  
首先测量每个训练实例如何与其最近的邻居线性相关，然后寻找可以最好的保留这些局部关系的训练集的低维表示形式  
适合展开扭曲的流形

In [15]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

# 8.6 其他降维技术

* *随机投影*
* *多维缩放(MDS)*
* *Isomap*
* *t分布随机近邻嵌入(t-SNE)*
* *线性判别分析(LDA)*